### Imports

In [1]:
from pylab import *
%matplotlib inline
rcParams['figure.figsize'] = (15.0, 30.0)

* Prepare paths

In [2]:
import sys
sys.path.append('..')

### General data structures

* Preload data

In [3]:
from dataset import get_positive_patches, get_negative_patches, get_labels, load_image
from common import dataset, utils
import numpy as np
import pandas as pd

labels_data, data = dataset.labels_data, dataset.data


## This one contains separatedly cars, vans...
#vehicle_types = set(dataset.map_category.values())
#vehicle_types.add('Empty')

map_category = {'A':'Vehicle', 'B':'Vehicle', 'C':'Vehicle', 'D':'Vehicle', 'E':'Vehicle', 'F':'Vehicle', 'G':'Vehicle', 'H':'Vehicle', 'I':'Vehicle'}
labels_data = pd.read_csv('../data/trainingObservations.csv')
data = get_labels(labels_data, map_category)
vehicle_types = set(map_category.values())

* Split train and test

In [4]:
#test_images = 60
#cases = list(data.keys())
#np.random.shuffle(cases)
#train_cases, test_cases = cases[test_images:], cases[:test_images]
#np.save('../data/patch_images/train_cases.npy', train_cases)
#np.save('../data/patch_images/test_cases.npy', test_cases)
train_cases = np.load('../data/patch_images/train_cases.npy')
test_cases  = np.load('../data/patch_images/test_cases.npy')

In [5]:
vehicle_types.add('Empty')

In [6]:
image_size_nn = 48

### Train a resnet50

In [ ]:
import os
import random
import logging
import multiprocessing
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from sklearn import metrics
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback
from dl_networks.resnet import ResnetBuilder
from dl_utils.tb_callback import TensorBoard


OUTPUT_MODEL = '../data/models/vehicle_empty_discriminator.hdf5'  # OUTPUT_MODEL = wp + 'personal/jm_patches_train_v06_local.hdf5'
LOGS_PATH    = '../data/logs/%s' % str('vehicle_empty_resnet_v0')
    
# Load model
K.set_image_dim_ordering('th')
model = ResnetBuilder().build_resnet_50((3,image_size_nn,image_size_nn),1)
model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])#,'fmeasure'])
model.load_weights(OUTPUT_MODEL)

Using TensorFlow backend.
../dl_networks/resnet.py:44: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(7, 7), filters=64, strides=(2, 2), padding="same", kernel_initializer="he_normal", kernel_regularizer=<keras.reg...)`
  init=init, border_mode=border_mode, W_regularizer=W_regularizer)(input)
../dl_networks/resnet.py:211: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")`
  pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), border_mode="same")(conv1)
../dl_networks/resnet.py:147: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(1, 1), filters=64, strides=(1, 1), padding="same", kernel_initializer="he_normal", kernel_regularizer=<keras.reg...)`
  W_regularizer=l2(0.0001))(input)
../dl_networks/resnet.py:65: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(3, 3), filters=64, strides=(1, 1), padding="same", kernel

### Test on an image

In [ ]:
import scipy
from scipy import ndimage
import matplotlib.pyplot as plt
from pylab import *
import time
import scipy.misc

def generate_patches(image, x0, x1, y0, y1, patch_size, sample_size, batch_size, grid_search_size = 10):
    ret_X = []
    for i in range(x0,x1-patch_size,grid_search_size):
        for j in range(y0, y1-patch_size,grid_search_size):
            a,b,c,d = i,i+patch_size,j,j+patch_size
            patch = image[a:b,c:d]
            resized_path = scipy.misc.imresize(patch, [sample_size, sample_size]) / 255
            #print(patch.max(), patch.min(), resized_path.min(), resized_path.max())
            ret_X.append(resized_path)
            if len(ret_X) == batch_size:
                yield np.array(ret_X).transpose([0,3,1,2])
                ret_X = []
    if len(ret_X) != 0:
        yield np.array(ret_X).transpose([0,3,1,2])

def run_scan_prediction(image, patch_size = 110, inp_net_size = 48, batch_size = 100, grid_search_size = 10):
    t1 = time.time()
    outs = []
    for x in generate_patches(image, 0, 2000, 0, 2000, patch_size, inp_net_size, batch_size, grid_search_size):
        outs.append(model.predict(x).flatten())
    outs = np.concatenate(outs)

    grid_size = int(np.sqrt(outs.shape[0]))
    outs = outs.reshape(grid_size,-1)
    print('It took: ', time.time()-t1, ' seconds')
    return outs

def identify_detected_cars(predicted_map, prediction_thrs, grid_search_size, patch_size, gaussian_filter_param, gaussian_threshold):
    binimg = (1-predicted_map)*(predicted_map < prediction_thrs)
    imgf = ndimage.gaussian_filter(binimg, gaussian_filter_param)
    threshold = gaussian_threshold

    # find connected components
    s = [[1,1,1],
         [1,1,1],
         [1,1,1]]
    
    labeled, nr_objects = ndimage.label(imgf > threshold, structure = s) 

    # Structure we use to calculate the centroid of each cluster
    index_matrix = np.array([[[i,j] for j in range(len(labeled))] for i in range(len(labeled))])

    detected_vehicles = []
    for i in range(1,nr_objects+1):
        try:
            x_mean = int(round(np.mean([x for x in (index_matrix[:,:,1] * (labeled == i)).flatten() if x != 0]) * grid_search_size + patch_size / 2))
            y_mean = int(round(np.mean([y for y in (index_matrix[:,:,0] * (labeled == i)).flatten() if y != 0]) * grid_search_size + patch_size / 2))
            size = (labeled==i).sum()
            detected_vehicles.append([x_mean,y_mean, size])
        except:
            print("An error happened while we were looking at detected object %d" % i)
    return detected_vehicles

def plot_map_and_coordinates(image, coordinates):
    figure()
    imshow(image)
    for x,y,s in coordinates:
        plot([x],[y], 'or')

def save_annotations(image_name, annotations_path, detected_cars):
    df = pd.DataFrame(columns = ['x','y'], data = {'x': [x[0] for x in detected_cars], 'y': [y[1] for y in detected_cars]})
    df.to_csv('%s/%s' % (annotations_path, image_name), header = None, index = None)
    
    
# General params
patch_size = 110
inp_net_size = 48

# Scanner params
grid_search_size = 20

# Car locator params
prediction_thrs = 0.4
gaussian_filter_param = 1.
gaussian_threshold = 0.2
annotations_path = '../data/patch_images/inferred_annotations'

for image_name in np.concatenate([test_cases, train_cases]):
    if image_name not in os.listdir(annotations_path):
        image = dataset.load_image(image_name)/255
        predicted_map = run_scan_prediction(image, patch_size = patch_size, inp_net_size = inp_net_size, grid_search_size = grid_search_size)
        detected_cars = identify_detected_cars(predicted_map, prediction_thrs, grid_search_size, patch_size, gaussian_filter_param, gaussian_threshold = gaussian_threshold)
        save_annotations(image_name, annotations_path, detected_cars)
        print("- %s calculated" % image_name)
    else:
        print("- %s already done" % image_name)

    #plot_map_and_coordinates(image, detected_cars)

- TQ3080_5_1 already done
- TQ2483_5_0 already done
- TQ3080_8_9 already done
- TQ2483_3_7 already done
- TQ3477_7_2 already done
- TQ3779_6_1 already done
- TQ3477_1_7 already done
- TQ2379_9_6 already done
- TQ2483_5_8 already done
- TQ2684_1_7 already done
- TQ2379_3_3 already done
- TQ3080_5_2 already done
- TQ2379_6_0 already done
- TQ3477_6_1 already done
- TQ2379_4_3 already done
- TQ2684_9_3 already done
- TQ2684_9_8 already done
- TQ3080_6_7 already done
- TQ2684_0_7 already done
- TQ3080_3_6 already done
- TQ2684_3_1 already done
- TQ2483_0_5 already done
- TQ3779_0_7 already done
- TQ2379_1_2 already done
- TQ2483_9_1 already done
- TQ2379_4_5 already done
- TQ2379_3_9 already done
- TQ2483_6_8 already done
- TQ2684_0_6 already done
- TQ2483_1_1 already done
- TQ2483_4_6 already done
- TQ2684_1_1 already done
- TQ3779_8_2 already done
- TQ3080_9_4 already done
- TQ2379_2_4 already done
- TQ3779_0_9 already done
- TQ2483_2_4 already done
- TQ3080_7_6 already done
- TQ3080_2_6